In [ ]:
import datetime
import matplotlib.dates as mdates
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from summer.utils import ref_times_to_dti

from autumn.settings import Region
from autumn.tools.curve.scale_up import scale_up_function
from autumn.tools.project import get_project, TimeSeriesSet
from autumn.tools.plots.utils import REF_DATE
from autumn.models.covid_19.constants import BASE_DATETIME, AGEGROUP_STRATA
from autumn.tools.calibration.targets import get_target_series
from autumn.tools.utils.utils import apply_moving_average
from autumn.tools.utils.pretty import pretty_print
from autumn.tools.inputs.covid_au.queries import get_both_vacc_coverage, get_vic_testing_numbers
from autumn.models.covid_19.detection import create_cdr_function
from autumn.tools.inputs.demography.queries import get_population_by_agegroup

Specify what we're going to run

In [ ]:
region = Region.VICTORIA
project = get_project("covid_19", region)
targets_path = f"../../../autumn/projects/covid_19/victoria/victoria/targets.secret.json"
ts_set = TimeSeriesSet.from_file(targets_path)
start_time = ref_times_to_dti(REF_DATE, [project.param_set.baseline["time"]["start"]])[0]

In [ ]:
new_params = {
    "contact_rate": 0.043, 
    "time": {"start": 525}, 
    "vaccination": {
        "program_start_time": 525, 
    },
}

In [ ]:
project.param_set.baseline = project.param_set.baseline.update(new_params)

Run the model

In [ ]:
model = project.run_baseline_model(project.param_set.baseline)
derived_df = model.get_derived_outputs_df()

Specify the outputs

In [ ]:
outputs = (
    "notifications", "infection_deaths", "hospital_admissions", "hospital_occupancy", "icu_admissions", "icu_occupancy"
)

Plot the outputs

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 12), sharex="all")
fig.tight_layout(w_pad=1.5, h_pad=3.5)
for i_ax, axis in enumerate(axes.reshape(-1)):
    output_name = outputs[i_ax]
    output = ts_set.get(output_name)
    axis.plot(ref_times_to_dti(REF_DATE, model.times), derived_df[outputs[i_ax]])
    axis.plot(derived_df.index, derived_df[outputs[i_ax]])
    axis.scatter(ref_times_to_dti(REF_DATE, output.times), output.values, c="k")
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_title(output_name)
    axis.set_xlim(left=start_time)

In [ ]:
# Get the CDR function of tests
cdr_from_tests_func = create_cdr_function(
    project.param_set.baseline["testing_to_detection"]["assumed_tests_parameter"],
    project.param_set.baseline["testing_to_detection"]["assumed_cdr_parameter"],
)

# Get the denominator population
testing_pops = get_population_by_agegroup(
    AGEGROUP_STRATA, project.param_set.baseline["country"]["iso3"], "Victoria", 
    year=project.param_set.baseline["population"]["year"]
)

# Process the data
test_times, test_values = get_vic_testing_numbers()
test_dates = ref_times_to_dti(BASE_DATETIME, [int(time) for time in test_times])
per_capita_tests = [i_tests / sum(testing_pops) for i_tests in test_values]
dummy_tests = np.linspace(0, max(per_capita_tests), 200)
if project.param_set.baseline["testing_to_detection"]["assumed_tests_parameter"]:
    smoothed_per_capita_tests = apply_moving_average(
        per_capita_tests, project.param_set.baseline["testing_to_detection"]["smoothing_period"]
    )
else:
    smoothed_per_capita_tests = per_capita_tests
    
cdr_function_of_time = scale_up_function(
    test_times,
    [cdr_from_tests_func(test_rate) for test_rate in smoothed_per_capita_tests],
    smoothness=0.2, method=4, bound_low=0.,
)    

# Plot
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.tight_layout(w_pad=1.5, h_pad=5)

# Plot daily number of tests
axis = axes[0, 0]
axis.plot(test_dates, test_values, marker="o")
axis.tick_params(axis="x", labelrotation=45)
axis.set_title("daily testing numbers")
axis.set_xlim(left=start_time)

# Plot daily number of tests
axis = axes[0, 1]
axis.plot(test_dates, per_capita_tests, label="raw")
axis.plot(test_dates, smoothed_per_capita_tests, label="smoothed")
axis.tick_params(axis="x", labelrotation=45)
axis.set_xlim(left=start_time)
axis.set_title("daily per capita testing rate")
axis.legend()

# Plot relationship of daily tests to CDR proportion
axis = axes[1, 0]
axis.plot(dummy_tests, cdr_from_tests_func(dummy_tests))
axis.scatter(per_capita_tests, [cdr_from_tests_func(i_tests) for i_tests in per_capita_tests], color="r")
axis.set_ylabel("case detection proportion")
axis.set_xlabel("per capita testing rate")
axis.set_title("daily per capita tests to CDR relationship")
axis.set_ylim([0., 1.])

# Plot CDR values
axis = axes[1, 1]
axis.scatter(test_dates, [cdr_from_tests_func(i_test_rate) for i_test_rate in smoothed_per_capita_tests], color="r")
axis.plot(test_dates, [cdr_function_of_time(time) for time in test_times])
axis.set_title("final case detection rate")
axis.set_ylabel("proportion")
axis.tick_params(axis="x", labelrotation=45)
axis.set_xlim(left=start_time)
axis.set_ylim([0., 1.])

fig.tight_layout()

## Inspect the computed value processes around contact tracing

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharex="all")
fig.tight_layout(w_pad=1.5, h_pad=3.5)

derived_df["prop_all_traced"] = derived_df["prop_contacts_with_detected_index"] * derived_df["prop_detected_traced"]

computed_processes = ("prevalence", "prop_contacts_with_detected_index", "prop_detected_traced", "prop_all_traced")
for i_ax, axis in enumerate(axes.reshape(-1)):
    process = computed_processes[i_ax]
    axis.plot(derived_df.index, derived_df[process])
    axis.set_title(process)
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=start_time)
    if i_ax > 0:
        axis.set_ylim((0., 1.))

In [ ]:
fig, axis = plt.subplots(1, 1, figsize=(12, 8), sharex="all")

axis.fill_between(
    derived_df.index, 
    [0.] * len(derived_df), 
    derived_df["proportion_fully_vaccinated"], 
    label="fully vaccinated"
)
axis.fill_between(
    derived_df.index, 
    derived_df["proportion_fully_vaccinated"], 
    derived_df["at_least_one_dose_prop"], 
    label="one dose only"
)
axis.fill_between(
    derived_df.index, 
    derived_df["at_least_one_dose_prop"], 
    [1.] * len(derived_df), 
    label="unvaccinated"
)
lower_age_limit = 0
vacc_times, vacc_coverage = get_both_vacc_coverage(region.upper(), start_age=lower_age_limit)
vacc_dates = ref_times_to_dti(BASE_DATETIME, [int(i) for i in vacc_times])  # Converting numpy ints returned
lagged_dates = ref_times_to_dti(BASE_DATETIME, [int(i) + 14 for i in vacc_times])
axis.plot(vacc_dates, vacc_coverage, color="k", label="actual vaccination")
axis.plot(lagged_dates, vacc_coverage, color="k", linestyle="--", label="lagged for immunity")
axis.tick_params(axis="x", labelrotation=45)
axis.set_title(region)
axis.legend()

In [ ]:
derived_df["prop_ever_infected"].plot(title="prop_ever_infected")